In [1]:
model_checkpoint = "google/vit-base-patch16-224-in21k"
model_checkpoint

'google/vit-base-patch16-224-in21k'

In [2]:
import os 
import torch
from peft import PeftModel, LoraConfig, get_peft_model
from transformers import AutoModelForImageClassification


def print_model_size(path):
    size = 0
    for f in os.scandir(path):
        size += os.path.getsize(f)

    print(f"Model size: {(size / 1e6):.2} MB")


def print_trainable_parameters(model, label):
    parameters, trainable = 0, 0
    
    for _, p in model.named_parameters():
        parameters += p.numel()
        trainable += p.numel() if p.requires_grad else 0

    print(f"{label} trainable parameters: {trainable:,}/{parameters:,} ({100 * trainable / parameters:.2f}%)")


def split_dataset(dataset):
    dataset_splits = dataset.train_test_split(test_size=0.1)
    return dataset_splits.values()
    

def create_label_mappings(dataset):
    label2id, id2label = dict(), dict()
    for i, label in enumerate(dataset.features["label"].names):
        label2id[label] = i
        id2label[i] = label 

    return label2id, id2label

/Users/jpv/Bharath_J_P_V/Personal_Projects/lora_fine_tuning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

# This is the food dataset
dataset1 = load_dataset("food101", split="train[:10000]")

# This is the datasets of pictures of cats and dogs.
# Notice we need to rename the label column so we can
# reuse the same code for both datasets.
# dataset2 = load_dataset("microsoft/cats_vs_dogs", split="train", trust_remote_code=True)
# dataset2 = dataset2.rename_column("labels", "label")

dataset1_train, dataset1_test = split_dataset(dataset1)
# dataset2_train, dataset2_test = split_dataset(dataset2)

Generating validation split: 100%|██████████| 25250/25250 [00:00<00:00, 28578.25 examples/s]


In [4]:
dataset1_label2id, dataset1_id2label = create_label_mappings(dataset1)
# dataset2_label2id, dataset2_id2label = create_label_mappings(dataset2)

In [8]:
model1 = {
        "train_data": dataset1_train,
        "test_data": dataset1_test,
        "label2id": dataset1_label2id,
        "id2label": dataset1_id2label,
        "epochs": 5,
        "path": "./lora-model1"
    }

In [10]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint, use_fast=True)

In [13]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    Resize,
    ToTensor,
)

preprocess_pipeline = Compose([
    Resize(image_processor.size["height"]),
    CenterCrop(image_processor.size["height"]),
    ToTensor(),
    Normalize(mean=image_processor.image_mean, std=image_processor.image_std),
])

def preprocess(batch):
    batch["pixel_values"] = [
        preprocess_pipeline(image.convert("RGB")) for image in batch["image"]
    ]
    return batch

In [14]:
model1["train_data"].set_transform(preprocess)
model1["test_data"].set_transform(preprocess)

In [16]:
import numpy as np
import evaluate
import torch
from peft import PeftModel, LoraConfig, get_peft_model
from transformers import AutoModelForImageClassification


metric = evaluate.load("accuracy")


def data_collate(examples):
    """
    Prepare a batch of examples from a list of elements of the
    train or test datasets.
    """
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


def compute_metrics(eval_pred):
    """
    Compute the model's accuracy on a batch of predictions.
    """
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


def get_base_model(label2id, id2label):
    """
    Create an image classification base model from
    the model checkpoint.
    """
    return AutoModelForImageClassification.from_pretrained(
        model_checkpoint,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,
    )


def build_lora_model(label2id, id2label):
    """Build the LoRA model to fine-tune the base model."""
    model = get_base_model(label2id, id2label)
    print_trainable_parameters(model, label="Base model")

    config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=["query", "value"],
        lora_dropout=0.1,
        bias="none",
        modules_to_save=["classifier"],
    )

    lora_model = get_peft_model(model, config)
    print_trainable_parameters(lora_model, label="LoRA")

    return lora_model


In [25]:
from transformers import TrainingArguments

batch_size = 16
training_arguments = TrainingArguments(
    output_dir="./model-checkpoints",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    use_cpu=True,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    label_names=["labels"],
)

In [26]:
from transformers import Trainer

training_arguments.num_train_epochs = model1["epochs"]

trainer = Trainer(
    build_lora_model(model1["label2id"], model1["id2label"]),
    training_arguments,
    train_dataset=model1["train_data"],
    eval_dataset=model1["test_data"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=data_collate,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Base model trainable parameters: 85,876,325/85,876,325 (100.00%)
LoRA trainable parameters: 667,493/86,543,818 (0.77%)


In [ ]:
from transformers import Trainer

for cfg in config.values():
    training_arguments.num_train_epochs = cfg["epochs"]

    trainer = Trainer(
        build_lora_model(cfg["label2id"], cfg["id2label"]),
        training_arguments,
        train_dataset=cfg["train_data"],
        eval_dataset=cfg["test_data"],
        tokenizer=image_processor,
        compute_metrics=compute_metrics,
        data_collator=data_collate,
    )

    results = trainer.train()
    evaluation_results = trainer.evaluate(cfg['test_data'])
    print(f"Evaluation accuracy: {evaluation_results['eval_accuracy']}")

    # We can now save the fine-tuned model to disk.
    trainer.save_model(cfg["path"])
    print_model_size(cfg["path"])

In [27]:
results = trainer.train()
evaluation_results = trainer.evaluate(model1['test_data'])
print(f"Evaluation accuracy: {evaluation_results['eval_accuracy']}")

# We can now save the fine-tuned model to disk.
trainer.save_model(model1["path"])
print_model_size(model1["path"])

  1%|▏         | 10/700 [01:22<1:32:34,  8.05s/it]

{'loss': 2.4884, 'grad_norm': 0.7463366389274597, 'learning_rate': 0.004928571428571429, 'epoch': 0.07}


  3%|▎         | 20/700 [02:42<1:30:46,  8.01s/it]

{'loss': 0.4479, 'grad_norm': 0.3320975601673126, 'learning_rate': 0.004857142857142858, 'epoch': 0.14}


  4%|▍         | 30/700 [04:03<1:29:51,  8.05s/it]

...

{'eval_loss': 0.20703613758087158, 'eval_accuracy': 0.946, 'eval_runtime': 50.3043, 'eval_samples_per_second': 19.879, 'eval_steps_per_second': 1.252, 'epoch': 4.97}


100%|██████████| 700/700 [1:50:43<00:00,  9.49s/it]


{'train_runtime': 6643.6853, 'train_samples_per_second': 6.773, 'train_steps_per_second': 0.105, 'train_loss': 0.13161656854408127, 'epoch': 4.97}


100%|██████████| 63/63 [00:48<00:00,  1.29it/s]


Evaluation accuracy: 0.946
Model size: 2.7 MB


In [28]:
def build_inference_model(label2id, id2label, lora_adapter_path):
    """Build the model that will be use to run inference."""

    # Let's load the base model
    model = get_base_model(label2id, id2label)

    # Now, we can create the inference model combining the base model
    # with the fine-tuned LoRA adapter.
    return PeftModel.from_pretrained(model, lora_adapter_path)


def predict(image, model, image_processor):
    """Predict the class represented by the supplied image."""
    
    encoding = image_processor(image.convert("RGB"), return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits

    class_index = logits.argmax(-1).item()
    return model.config.id2label[class_index]


In [29]:
model1["inference_model"] = build_inference_model(model1["label2id"], model1["id2label"], model1["path"]) 
model1["image_processor"] = AutoImageProcessor.from_pretrained(model1["path"])

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
images = {
    'image1': "https://www.allrecipes.com/thmb/AtViolcfVtInHgq_mRtv4tPZASQ=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/ALR-187822-baked-chicken-wings-4x3-5c7b4624c8554f3da5aabb7d3a91a209.jpg",
    'image2':  "https://www.simplyrecipes.com/thmb/KE6iMblr3R2Db6oE8HdyVsFSj2A=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/__opt__aboutcom__coeus__resources__content_migration__simply_recipes__uploads__2019__09__easy-pepperoni-pizza-lead-3-1024x682-583b275444104ef189d693a64df625da.jpg"
}

In [32]:
from PIL import Image
import requests

image = Image.open(requests.get(images["image2"], stream=True).raw)

inference_model = model1["inference_model"]
image_processor = model1["image_processor"]

prediction = predict(image, inference_model, image_processor)
print(f"Prediction: {prediction}")

Prediction: pizza
